In [ ]:
from myosuite.utils import gym
import numpy as np

In [ ]:
from myosuite.envs.myo.myochallenge.relocate_v0 import RelocateEnvV0

class myEnv(RelocateEnvV0):
    def get_reward_dict(self, obs_dict):
        reach_dist = np.abs(np.linalg.norm(self.obs_dict['reach_err'], axis=-1))
        obj_pos_z =  self.obs_dict['obj_pos'][:,:,2] - np.array(0.9283)
        pos_dist = np.abs(np.linalg.norm(self.obs_dict['pos_err'], axis=-1))
        rot_dist = np.abs(np.linalg.norm(self.obs_dict['rot_err'], axis=-1))
        act_mag = np.linalg.norm(self.obs_dict['act'], axis=-1)/self.sim.model.na if self.sim.model.na !=0 else 0

        rwd_dict = super().get_reward_dict(obs_dict)

        rwd_dict['done'] = np.greater(obs_dict['time'], 8)

        rwd_dict['dense'] = -5*reach_dist - 1*act_mag + 4*obj_pos_z

        return rwd_dict


env = myEnv(model_path="C:/Users/Tom/miniconda3/envs/myosuite/Lib/site-packages/myosuite/envs/myo/assets/arm/myoarm_relocate.xml",
            target_xyz_range = {'high':[0.2, -.1, 0.9], 'low':[0.0, -.35, 0.9]},
            target_rxryrz_range = {'high':[0.0, 0.0, 0.0], 'low':[0.0, 0.0, 0.0]})


env.reset();


In [ ]:
from stable_baselines3 import PPO

model = PPO("MlpPolicy", env, verbose=1)

In [ ]:
model = PPO.load("TrainedModels/Manipulate_model_new2", env=env, device='cpu')


In [ ]:
print("========================================")
print("Starting policy learning")
print("========================================")

env.reset();
env.mj_render()
model.learn(total_timesteps=400000)

print("========================================")
print("Job Finished.") 
print("========================================")

model.save('TrainedModels/Manipulate_model_new3')


In [ ]:
from time import sleep

env.reset()

for _ in range(10):
    for _ in range(300):
        env.mj_render()
        obs = env.get_obs()
        action = model.predict(obs)[0]
        env.step(action)
        sleep(0.01)
    
    env.reset()

env.close()
